In [5]:
import pandas as pd
import os
import re

In [238]:
data = pd.read_csv("/home/nmh/ds/project/data_cleaning_batdongsanso/batdongsanso/batdongsanso_concat.csv")
data.head()

,price,address,Diện tích,Hướng,Mặt tiền,Lộ giới,Số tầng,Số phòng ngủ,Số toilet,Details,Ngày đăng,Loại tin,id
0,"10,9 tỷ VNĐ","Cau Giay, Ha Noi",78m2,Đông nam,3.0,5.0,7.0,10.0,10.0,DÒNG TIỀN LÊN TỚI 1 TỶ/NĂM – GẦN MẶT PHỐ – NG...,21/02/2022,Tin thường,batdongsan_so_com_vn_1673831404.5091681
1,14 tỷ VNĐ,"Láng Ha, Đống Đa, Hà Nội.",50m2,Đông nam,5.0,NaN,3.0,NaN,NaN,KHU CỰC VIP - LÔ GÓC - Ô TÔ TRÁNH VỈA HÈ - KIN...,22/02/2022,Tin thường,batdongsan_so_com_vn_1673831405.9842184
2,"14,9 tỷ VNĐ","Giang Văn Minh , Kim Mã , Ba Đình, Hà Nôi.",88m2,NaN,3.0,NaN,6.0,10.0,10.0,PHÂN LÔ TRUNG TÂM THỦ ĐÔ - NGÕ 4 CHỖ - DÂN TRÍ...,23/02/2022,Tin thường,batdongsan_so_com_vn_1673831407.2727795
3,"5,89 tỷ VNĐ",Hà Nội,40m2,NaN,4.0,NaN,6.0,10.0,10.0,"KHU VỰC KINH DOANH DÒNG TIỀN BẠT NGÀN, LUÔN FU...",28/02/2022,Tin thường,batdongsan_so_com_vn_1673831408.4476721
4,"2,87 tỷ VNĐ",Hà Nội,25m2,NaN,3.0,NaN,4.0,3.0,NaN,"NHỎ XINH THOÁNG MÁT, Ô TÔ CÁCH 10M, TẦM TIỀN. ...",01/03/2022,Tin thường,batdongsan_so_com_vn_1673831409.5217986


In [239]:
unique_values = data['Hướng'].unique()
print(unique_values)

[' Đông nam' nan ' Đông bắc' ' Nam' ' Tây nam' ' Tây bắc' ' Tây' ' Đông'
 ' Bắc']


In [6]:
def extract_address_components(address):
    if not isinstance(address, str):
        return ["", "", ""]

    address = address.lower()

    patterns_to_remove = ['hà nội', 'thành phố hà nội', 'tp hà nội',',hà nội', ', hà nội', ', thành phố hà nội', ', tp hà nội', '- hà nội', '- thành phố hà nội', '- tp hà nội']
    
    for pattern in patterns_to_remove:
        address = address.replace(pattern, '')

    # Split address into parts
    parts = re.split(r',\s*|\-\s*', address)
    
    # Initialize default values
    street, ward, district = "", "", ""
    if len(parts) == 1 and not any(kw in parts[0].lower() for kw in ['đường', 'phường', 'quận', 'huyện', 'xã', 'phố', 'thị trấn']):
        return [parts[0], "", ""]
    if len(parts) == 3:
        print(parts)
        return parts
    elif len(parts) == 2:
        return ["", parts[0], parts[1]]
    else:
        hanoi_districts = ['Hoàn Kiếm', 'Đống Đa', 'Ba Đình', 'Hai Bà Trưng', 'Hoàng Mai', 'Thanh Xuân', 'Long Biên', 'Nam Từ Liêm', 'Bắc Từ Liêm', 'Tây Hồ', 'Cầu Giấy', 'Hà Đông', 'Sơn Tây', 'Ba Vì', 'Chương Mỹ', 'Phúc Thọ', 'Đan Phượng', 'Đông Anh', 'Gia Lâm', 'Hoài Đức', 'Mê Linh', 'Mỹ Đức', 'Phú Xuyên', 'Quốc Oai', 'Sóc Sơn', 'Thạch Thất', 'Thanh Oai', 'Thường Tín', 'Ứng Hòa', 'Thanh Trì']
        for d in hanoi_districts:
            if d.lower() in address:
                district = d
                break
        street_pattern = r'(đường|phố|thị trấn)\s*([^,(phường|xã)]+)'
        ward_pattern = r'(phường|xã)\s*([^,]+)'
        district_pattern = r'(quận|huyện)\s*([^,]+)'

        # Extract ward first
        ward_match = re.search(ward_pattern, address, re.IGNORECASE)
        if ward_match:
            ward = ward_match.group(1).strip()
            # Update street pattern if ward is found
            # street_pattern = r'Đường\s*([^,]+?)(?=, Phường| Phường)'

        # Extract street
        street_match = re.search(street_pattern, address, re.IGNORECASE)
        if street_match:
            street = street_match.group(1).strip()

        # Extract district if it follows the ward, or take the last part of the address
        # district_match = re.search(district_pattern, address, re.IGNORECASE)
        # if district_match:
        #     district = district_match.group(1).strip()
        # elif len(parts) > 3:
        #     district = parts[-1].strip()

    return [street, ward, district]


In [3]:
def extract_address(address):
    if not isinstance(address, str):
        return ["", "", ""]

    address = address.lower()
    # print(address)

    patterns_to_remove = [", hà nội",', hà nội',', hà nội' ', thành phố hà nội', ', tp hà nội', '- hà nội', '- thành phố hà nội', '- tp hà nội', 'hà nội', 'thành phố hà nội', 'tp hà nội', ]
    
    for pattern in patterns_to_remove:
        address = address.replace(pattern, '')

    # print(address)
    parts = re.split(r',\s*|[-–—]\s*', address)
    # print(parts)

    # List of districts for matching
    hanoi_districts = ['Hoàn Kiếm', 'Đống Đa', 'Ba Đình', 'Hai Bà Trưng', 'Hoàng Mai', 
                       'Thanh Xuân', 'Long Biên', 'Nam Từ Liêm', 'Bắc Từ Liêm', 'Tây Hồ', 
                       'Cầu Giấy', 'Hà Đông', 'Sơn Tây', 'Ba Vì', 'Chương Mỹ', 'Phúc Thọ', 
                       'Đan Phượng', 'Đông Anh', 'Gia Lâm', 'Hoài Đức', 'Mê Linh', 'Mỹ Đức', 
                       'Phú Xuyên', 'Quốc Oai', 'Sóc Sơn', 'Thạch Thất', 'Thanh Oai', 
                       'Thường Tín', 'Ứng Hòa', 'Thanh Trì']
    
    district_found = ""
    for d in hanoi_districts:
        if d.lower() in address:
            district_found = d
            print(district_found)
            break
    if len(parts) == 3:
        street, ward, district = parts
    elif len(parts) == 2:
        street, ward = "", parts[0]
        district = parts[1]
    else:
        for part in parts:
    # Regular expressions for street and ward
            street_pattern = r'(đường|phố|thị trấn)\s*([^,]+?)(?=[-–—]|,| phường| xã|$)'
            ward_pattern = r'(phường|xã)\s*([^,]+)'

    # Extract street and ward
            street_match = re.search(street_pattern, address, re.IGNORECASE)
            ward_match = re.search(ward_pattern, address, re.IGNORECASE)

            street = street_match.group(2).strip().capitalize() if street_match else ""
            ward = ward_match.group(2).strip().capitalize() if ward_match else ""
            district = district_found.capitalize()

    return [street, ward, district]

In [2]:
s = "Giáp Bát, Hoàng Mai, Hà Nội"
patterns_to_remove = ['Hà Nội', 'hà nội', '- thành phố hà nội',"thành phố hà nội", ', tp hà nội', '- hà nội', '- thành phố hà nội', '- tp hà nội', 'hà nội', 'thành phố hà nội', 'tp hà nội']
def test(s):
    s = s.lower()
    print(s)
    for pattern in patterns_to_remove:
        s = s.replace(pattern, "")
    s.rstrip(s[-1])
    s.strip()
    print(s)
    return s
print(test(s))

giáp bát, hoàng mai, hà nội
giáp bát, hoàng mai, hà nội
giáp bát, hoàng mai, hà nội


In [7]:
addresss = extract_address("Giáp Bát, Hoàng Mai, Hà Nội")
address_ = extract_address("Giáp Bát, Hoàng Mai, Hà Nội")
print(addresss)
print(address_)

Hoàng Mai
['', 'giáp bát', 'hoàng mai']
['', 'giáp bát', 'hoàng mai']


In [245]:
def convert_price(price_str, area):
    if not isinstance(price_str, str):
        return None
    price_str = price_str.replace(',', '.').strip()
    if '/tháng' in price_str:
        return None
    if 'tỷ' in price_str:
        price_str = price_str.replace(' tỷ VNĐ', '').replace(' tỷ', '')
        try:
            return float(price_str) * 1000
        except ValueError:
            return None
    elif 'triệu' in price_str:
        try:
            return float(price_str.replace(' triệu VNĐ', '').replace(' triệu', ''))
        except ValueError:
            return None
    elif '/m2' in price_str:
        try:
            price_per_m2 = float(price_str.replace('/m2', ''))
            return price_per_m2 * area
        except ValueError:
            return None
    else:
        return None

In [246]:
def clean_direction(direction_str):
    direction_map = {
        'đông': 'E', 'tây': 'W', 'nam': 'S', 'bắc': 'N',
        'tây bắc': 'WN', 'tây nam': 'WS', 'đông bắc': 'EN', 'đông nam': 'ES'
    }
    direction_str = direction_str.lower().strip() if isinstance(direction_str, str) else direction_str
    return direction_map.get(direction_str, "")

In [247]:
def determine_furniture(details):
    if isinstance(details, str) and 'nội thất' in details:
        return 'Available'
    return 'None'

In [248]:
def clean_legitimacy(details):
    if isinstance(details, str):
        details_lower = details.lower()
        if 'sổ đỏ' in details_lower:
            return 'Sổ đỏ'
        elif 'sổ hồng' in details_lower:
            return 'Sổ hồng'
        elif 'chưa có sổ' in details_lower or 'chưa sổ' in details_lower:
            return 'None'
        elif 'sổ' in details_lower:
            return 'Đã có sổ'
    return 'None'

In [249]:
folder_path = '/home/nmh/ds/project/data/batdongsanso'
all_dfs = []

In [250]:
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)

        df[['street', 'ward', 'district']] = df['address'].fillna("").apply(lambda x: pd.Series(extract_address(x)))
        df['city'] = 'Hà Nội'

        df['area'] = df['Diện tích'].str.replace('m2', '').str.strip().astype(float).fillna(0)

        df['price'] = df.apply(lambda row: convert_price(row['price'], row['area']), axis=1)

        if 'Hướng' in df.columns and df['Hướng'].notnull().any():
            df['house_direction'] = df['Hướng'].apply(clean_direction)
        else:
            df['house_direction'] = None

        if 'Số phòng ngủ' in df.columns:
            df['bedroom'] = pd.to_numeric(df['Số phòng ngủ'], errors='coerce')
        else:
            df['bedroom'] = None

        if 'Số toilet' in df.columns:
            df['toilet'] = pd.to_numeric(df['Số toilet'], errors='coerce')
        else:
            df['toilet'] = None

        if 'Số tầng' in df.columns:
            df['floors'] = pd.to_numeric(df['Số tầng'], errors='coerce')
        else:
            df['floors'] = None

        if 'Ngày đăng' in df.columns:
            df['upload_date'] = df['Ngày đăng']
        else:
            df['date'] = None

        df['furniture'] = df['Details'].apply(determine_furniture)
        df['legits'] = df['Details'].apply(clean_legitimacy)

        df['facade'] = pd.to_numeric(df['Mặt tiền'], errors='coerce') if 'Mặt tiền' in df.columns else None
        df['entrance'] = pd.to_numeric(df['Lộ giới'], errors='coerce') if 'Lộ giới' in df.columns else None

        df = df[df['price'].notnull()]

        final_columns = ['id', 'city', 'district', 'ward', 'street', 'area', 'price', 'house_direction', 
                         'bedroom', 'toilet', 'floors', 'legits','upload_date', 'furniture', 
                         'facade', 'entrance']
        df_final = df[final_columns]
        all_dfs.append(df_final)

đường trần quốc vượng, dịch vọng hậu, cầu giấy
đường trần quốc vượng, dịch vọng hậu, cầu giấy
['đường trần quốc vượng', 'dịch vọng hậu', 'cầu giấy']
Cầu Giấy
tân mai
tân mai
['tân mai']
đường nguyễn khang, trung hòa, cầu giấy
đường nguyễn khang, trung hòa, cầu giấy
['đường nguyễn khang', 'trung hòa', 'cầu giấy']
Cầu Giấy
đường đại la, đồng tâm, hai bà trưng
đường đại la, đồng tâm, hai bà trưng
['đường đại la', 'đồng tâm', 'hai bà trưng']
Hai Bà Trưng
đường láng thượng, láng thượng, đống đa
đường láng thượng, láng thượng, đống đa
['đường láng thượng', 'láng thượng', 'đống đa']
Đống Đa
đường thượng thanh, thượng thanh, long biên
đường thượng thanh, thượng thanh, long biên
['đường thượng thanh', 'thượng thanh', 'long biên']
Long Biên
đường chùa láng, láng thượng, đống đa
đường chùa láng, láng thượng, đống đa
['đường chùa láng', 'láng thượng', 'đống đa']
Đống Đa
đường hoàng cầu, ô chợ dừa, đống đa
đường hoàng cầu, ô chợ dừa, đống đa
['đường hoàng cầu', 'ô chợ dừa', 'đống đa']
Đống Đa
đư

villa 2 - 01 hyundai hill state, hà cầu, hà đông, hà nội
villa 2 - 01 hyundai hill state, hà cầu, hà đông
['villa 2 ', '01 hyundai hill state', 'hà cầu', 'hà đông']
Hà Đông
ngõ 98/20 tân triều - triều khúc, thanh trì, hà nội
ngõ 98/20 tân triều - triều khúc, thanh trì
['ngõ 98/20 tân triều ', 'triều khúc', 'thanh trì']
Thanh Trì
thôn khoang mãi, xã đồng trúc
thôn khoang mãi, xã đồng trúc
['thôn khoang mãi', 'xã đồng trúc']
hà nội

['']
kđt văn quán, hà đông, hà nội.
kđt văn quán, hà đông.
['kđt văn quán', 'hà đông.']
Hà Đông
phố nguyễn ngọc nại, phường khương trung
phố nguyễn ngọc nại, phường khương trung
['phố nguyễn ngọc nại', 'phường khương trung']
25 lê văn lương
25 lê văn lương
['25 lê văn lương']
hà nội

['']
đường lê hồng phong, hà cầu, hà đông, hà nội.
đường lê hồng phong, hà cầu, hà đông.
['đường lê hồng phong', 'hà cầu', 'hà đông.']
Hà Đông
phố tràng tiền, phường tràng tiền
phố tràng tiền, phường tràng tiền
['phố tràng tiền', 'phường tràng tiền']
đường tây trà- lĩnh nam
đư

In [251]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19 entries, 0 to 19
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               19 non-null     object 
 1   city             19 non-null     object 
 2   district         19 non-null     object 
 3   ward             19 non-null     object 
 4   street           19 non-null     object 
 5   area             19 non-null     float64
 6   price            19 non-null     float64
 7   house_direction  0 non-null      object 
 8   bedroom          14 non-null     float64
 9   toilet           8 non-null      float64
 10  floors           14 non-null     float64
 11  legits           19 non-null     object 
 12  upload_date      19 non-null     object 
 13  furniture        19 non-null     object 
 14  facade           14 non-null     float64
 15  entrance         10 non-null     float64
dtypes: float64(7), object(9)
memory usage: 2.5+ KB


In [252]:
combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df.to_csv('cleaned_data.csv', index=False)

In [253]:
none_or_empty_counts = combined_df.apply(lambda x: x.isna() | (x == '')).sum()
print(none_or_empty_counts)

id                     0
city                   0
district            9136
ward                5770
street              9941
area                   0
price                  0
house_direction    18609
bedroom             8434
toilet             10640
floors              6392
legits                 0
upload_date            0
furniture              0
facade              7780
entrance           10537
dtype: int64


In [254]:
for column in combined_df.columns:
    print(f"Column: {column}")
    print(combined_df[column].head())
    print("\n")

Column: id
0    batdongsan_so_com_vn_1673787773.2450073
1    batdongsan_so_com_vn_1673787776.9490986
2    batdongsan_so_com_vn_1673787779.6528642
3    batdongsan_so_com_vn_1673787780.9154882
4    batdongsan_so_com_vn_1673787782.2050369
Name: id, dtype: object


Column: city
0    Hà Nội
1    Hà Nội
2    Hà Nội
3    Hà Nội
4    Hà Nội
Name: city, dtype: object


Column: district
0        cầu giấy
1                
2        cầu giấy
3    hai bà trưng
4         đống đa
Name: district, dtype: object


Column: ward
0    dịch vọng hậu
1                 
2        trung hòa
3        đồng tâm
4      láng thượng
Name: ward, dtype: object


Column: street
0    đường trần quốc vượng
1                         
2       đường nguyễn khang
3             đường đại la
4        đường láng thượng
Name: street, dtype: object


Column: area
0     45.0
1     85.0
2    250.0
3     80.0
4     45.0
Name: area, dtype: float64


Column: price
0     5100.0
1     2350.0
2      200.0
3    10200.0
4     5500.0
Name: 

In [255]:
missing_percentages = combined_df.isnull().mean() * 100

print("Percentage of Missing Values per Column:")
print(missing_percentages)

Percentage of Missing Values per Column:
id                  0.000000
city                0.000000
district            0.000000
ward                0.000000
street              0.000000
area                0.000000
price               0.000000
house_direction    34.237881
bedroom            42.148926
toilet             53.173413
floors             31.944028
legits              0.000000
upload_date         0.000000
furniture           0.000000
facade             38.880560
entrance           52.658671
dtype: float64
